# Titanic Survival Analysis

Starter notebook for exploring the Kaggle Titanic dataset.

Planned sections:
- Load data
- Clean and preprocess
- Exploratory data analysis (EDA)
- Simple baseline model
- Results and next steps
